In [1]:
!pwd

/Users/ethan/code/note_llm/traffic


In [2]:
import pandas as pd
import numpy as np

def generate_traffic_data(num_samples=1000, seq_len=24, pred_len=6):
    """生成模拟交通流量时间序列数据"""
    timestamps = pd.date_range(start="2024-01-01", periods=num_samples+seq_len+pred_len, freq="H")
    data = {
        "time": [],
        "road_id": [],
        "traffic_flow": [],
        "prompt": [],
        "target": []
    }
    
    for i in range(num_samples):
        # 生成路段ID和基础流量
        road_id = np.random.choice(["A1", "B2", "C3"])
        base_flow = np.random.randint(50, 200)
        
        # 生成带周期性的时间序列
        historical = base_flow + 20 * np.sin(np.linspace(0, 2*np.pi, seq_len)) + np.random.normal(0, 5, seq_len)
        future = base_flow + 20 * np.sin(np.linspace(2*np.pi, 4*np.pi, pred_len)) + np.random.normal(0, 5, pred_len)
        
        # 构建Prompt
        prompt = f"已知过去{seq_len}小时交通流量为：{historical.tolist()}，预测未来{pred_len}小时流量为："
        target = ", ".join(map(str, future.round().astype(int)))
        
        # 填充数据
        data["time"].append(timestamps[i:i+seq_len])
        data["road_id"].append(road_id)
        data["traffic_flow"].append(historical.tolist())
        data["prompt"].append(prompt)
        data["target"].append(target)
    
    return pd.DataFrame(data)

# 生成并保存数据
traffic_df = generate_traffic_data()
traffic_df.to_csv("traffic_dataset.csv", index=False)

/var/folders/hz/k38sdngd20sbs7s5hf8l37tw0000gn/T/ipykernel_83493/2398609037.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  timestamps = pd.date_range(start="2024-01-01", periods=num_samples+seq_len+pred_len, freq="H")


In [5]:
from datasets import Dataset

# Convert the 'time' column to a list of strings
traffic_df['time'] = traffic_df['time'].apply(lambda x: [str(t) for t in x])

# Create the Dataset
dataset = Dataset.from_pandas(traffic_df)
dataset = dataset.train_test_split(test_size=0.2)

In [10]:
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer
login()
model_name = "Qwen/Qwen2-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
